In [ ]:
# Téléchagement

! pip install transformers

     |████████████████████████████████| 2.1MB 6.3MB/s 
     |████████████████████████████████| 3.3MB 21.4MB/s 
     |████████████████████████████████| 901kB 41.8MB/s 


# Préparation des données

In [ ]:
# Chargement des données
import pandas as pd
import numpy as np

df0 = pd.read_csv('https://raw.githubusercontent.com/MARCTELLY/NLP-TER/main/clean_donaldTrump_tweets%20-%20clean_donaldTrump_tweets.csv')
df0 = df0[df0['is_retweet']=="FALSE"]
df0['text'].to_csv('twetts.txt')

In [ ]:
# Lecture du fichier texte
text_list = df0[['text']].select_dtypes(exclude=[np.float]).dropna(axis=0)
text = [tweet for tweet in text_list.text]

In [ ]:
text[:5]

['71000000 Legal Votes. The most EVER for a sitting President!',
 'THE OBSERVERS WERE NOT ALLOWED INTO THE COUNTING ROOMS. I WON THE ELECTION GOT 71000000 LEGAL VOTES. BAD THINGS HAPPENED WHICH OUR OBSERVERS WERE NOT ALLOWED TO SEE. NEVER HAPPENED BEFORE. MILLIONS OF MAIL-IN BALLOTS WERE SENT TO PEOPLE WHO NEVER ASKED FOR THEM!',
 'I WON THIS ELECTION BY A LOT!',
 'Georgia Counties Using Same Software as Michigan Counties Also Encounter ‘Glitch’  via @BreitbartNews What a total mess this “election” has been!',
 'Lawyer’s Press Conference at Four Season’s Landscaping Philadelphia. Enjoy!']

In [ ]:
from sklearn.model_selection import train_test_split

tweet_train, tweet_test = train_test_split(text, test_size=.1)

with open("tweet_test.txt", 'w') as f:
  f.write(str(tweet_text))


with open("tweet_train.txt", 'w') as f:
  f.write(str(tweet_train))

# Modèle

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
train_path = "tweet_train.txt"
test_path = "tweet_test.txt"

In [ ]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
from transformers import Trainer, TrainingArguments

model = GPT2LMHeadModel.from_pretrained("gpt2")


training_args = TrainingArguments(
    output_dir="./gpt2-tweet", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model()

In [ ]:
from transformers import pipeline

tweet = pipeline('text-generation',model='./gpt2-tweet', tokenizer='gpt2',config={'max_length':250})




In [ ]:
tweet("barack obama")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'barack obama is trying to take away our National Anthem", \'Thank you!\', \'Just talked to @BarackObama about his "Biggest Victory" with the African American Community. It is the biggest American achievement yet and it will be'}]

In [ ]:
tweet("Democrates are")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Democrates are a disgrace to the Republican Party and the Country', 'A major victory for the @WhiteHouse for the people!', 'Congressman @David_Hagel for his leadership and commitment to @thehill is a hero for our"}]

In [ ]:
tweet("Thank to our")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Thank to our beautiful @GOPleaders…', 'RT @TomFitton: “Senator Mark Warner did an amazing job on Fox &amp; Friends last night. Great job on your new book. You are the man for it but you still"}]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import pipeline

tweet = pipeline('text-generation',model='/content/drive/MyDrive/TER/gpt-tweet', tokenizer='gpt2',config={'max_length':128})

In [ ]:
tweet("Barack Obama and democrats")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Barack Obama and democrats can’t get their way...we’re at a standstill', 'Congressman Mike Crapo (@CumpoGOP) has been an incredible advocate for America and has my Complete and Total End"}]

In [ ]:
tweet("Joe")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Joe.  My time is going to come when I’m 100% there and you will WIN!\', \'@fantasticlouis: Best conversation ever with Donald Trump!  He\'ll talk about golf!", \'Thank you for'}]

In [ ]:
tweet("sleepy joe")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'sleepy joe - is this what you really think?"  I don\'t have anything else. @realDonaldTrump!", \'If our @FITZ President @realDonaldTrump runs you will win.  Great.\', \'RT @GOPChairwoman: As'}]

In [ ]:
tweet("sleepy Joe")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'sleepy Joe Biden? He is also a totally stupid person!\', "Donald Trump\'s approval rating in the RealClearPolitics Poll is at a record low - he\'s the only one who can make it.", \'@_T_Leigh @'}]

In [ ]:
tweet("MAKE")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'MAKE AMERICA GREAT AGAIN!,11-22-2014 02:59:11,14,false,36535267713672907\\nTwitter for Android,Just finished my great book "How to Make Life More So" by'}]

In [ ]:
tweet("Hillary")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Hillary Clinton was also caught asking how many women are in her party?", \'@jakkarlie:  @realDonaldTrump @the_glennfj  He is my mentor I love him and I know he will make this country great again'}]